## __Random Initial Mass Fraction Generator__

For the training, testing and validation of our ANN we need to generate a proper data set in such a way that the network can perform supervised learning from it, while keeping grounded on the physics of reality.

In lay terms, this means that it would not make sense to train a network to predict values that are not possible in real life. The intended way to generate the data samples is to solve the system of stiff ordinary differential equations and use the evolution steps of the numerical solver as _input_ and _label_ sets. For this we need to provide first a set of random initial conditions (in this case initial mass fractions and temperature) to pass to an ODE solver. 

It is important that these randomly generated initial conditions agree with was is physically possible. Mathematically speaking, it is possible to generate an infinite number of initial mass fractions containing all the species involved in a given reaction mechanism, however, the existence of some of these might not be actually plausible since they could not agree with one or more conservation law. In particular we'd like our samples to comply with mass conservation.

One way this is feasible is by using Bilger's Definition for a mixture. 

## Setting the work environment
The libraries that will be used for this are:

    -Cantera
    -molmass
    -Numpy

In [36]:
import numpy as np
from molmass import Formula
import cantera as ct

#Set printing format for numpy arrays for ease of readin
np.set_printoptions(formatter={'float': lambda x: "{0:0.4f}".format(x)})

## Analysis of the combustion gas

Before proceeding to generating the set of initial random condition we need to be able to obtain the chemical properties of our gas mixture. The library _Cantera_ is useful in this endeavor since it allows for the reading of information from a reaction mechanism and then provides functionality for interactive with them in Python.

In [2]:
# We read a solution mechanism and assign its properties to an object
gas = ct.Solution('Mechanisms/WD_Laubscher.cti')

After assigning the properties of a reaction mechanism to an object, _Cantera_ provides functions to use them.

In [66]:
#Examples of some of the Cantera functions to be used:


elements=gas.element_names #Elements present in the mechanism
species=gas.species_names #Species present in the mechanism
print(' Elements in the mechanism:',elements,'\n',
     'Species in the mechanism:',species,'\n',
     'Number of atoms of Oxygen in Methane:'
      ,gas.n_atoms('CH4','O'),'\n',#Number of atoms of an element in a given species
     'Number of atoms of Oxygen in Water:'
      ,gas.n_atoms('H2O','O'),'\n',
      'Mass of Hydrogen: ', #Atomic weight of a given element present in the mechanism
      gas.atomic_weight('H'),'\n',
      'Mass of Methane: ', #Molecular weight of a given species present in the mechanism
      gas['CH4'].molecular_weights[0]
     )


 Elements in the mechanism: ['O', 'H', 'C', 'N', 'Ar'] 
 Species in the mechanism: ['CH4', 'O2', 'H2O', 'N2', 'CO', 'CO2', 'H2'] 
 Number of atoms of Oxygen in Methane: 0.0 
 Number of atoms of Oxygen in Water: 1.0 
 Mass of Hydrogen:  1.00794 
 Mass of Methane:  16.04276


Since noble gases do not interact in any reaction (footnote), the following function is designed to remove them:

In [4]:
#Noble gases filter:
def noble_filter(elements):
    filter_elements=[]
    # List of elements to be removed
    noble=['Ar','Kr','Ne','He','Rn','Xe']
    for x in elements:
        if (noble.count(x)==0):
            filter_elements.append(x)
            
    return filter_elements

In [44]:
elements=noble_filter(gas.element_names)
print('Elements without noble gases: ',elements)

Elements without noble gases:  ['O', 'H', 'C', 'N']


Using the functions described above is possible to build one that generates the matrix needes for the linear system of equations to be solved in order for the initial conditions to agree with Bilger's Definition.

## Bilger's Definition

Using the functions described above is possible to build one that generates the matrix needes for the linear system of equations to be solved in order for the initial conditions to agree with Bilger's Definition.

In [37]:
#Matrix for the linear system of Equations:
def bilger_matrix(gas):
    
    #Elements present in the mechanism
    elements=noble_filter(gas.element_names)
    #Species present in the mechanism
    species=gas.species_names
    
    #Matrix to store coeffiecients
    A=np.ones((len(elements)+1,len(species)))
   
    row=1;
    for x in elements:
        col=0
        for y in species:
            A[row,col]=gas.n_atoms(y,x)*gas.atomic_weight(x)/gas[y].molecular_weights[0]
            col +=1
        row +=1
    return np.copy(A)

A=bilger_matrix(gas)
print(A)

[[1.0000 1.0000 1.0000 1.0000 1.0000 1.0000 1.0000]
 [0.0000 1.0000 0.8881 0.0000 0.5712 0.7271 0.0000]
 [0.2513 0.0000 0.1119 0.0000 0.0000 0.0000 1.0000]
 [0.7487 0.0000 0.0000 0.0000 0.4288 0.2729 0.0000]
 [0.0000 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000]]


As for the resulting vector, it will be created using random mixture fractions of a fuel and an oxydizer. Python dictionaries bestow a simple way to storing this information. For the results to be reproducible, it is necessary to use a seeded random number generator. Nonetheles the implementation of it won't come until later.

In [55]:
#Dictionaries for oxydizer and fuel mass fractions:
fuel={'H':0.03922422,'O':0.19663877,'C':0.11684286,'N':0.64729}
oxy={'H':1.03625e-5,'O':0.23298543,'C':3.194117e-5,'N':0.766970811}

def bilger_vector(f_mix, fuel_dict, oxy_dict, elements):
    
    b=np.ones(len(elements)+1)
    row=1
    for x in elements:
        b[row]=f_mix*fuel_dict[x]+(1-f_mix)*oxy_dict[x]
        row+=1
    return np.copy(b)

In [56]:
np.random.seed(0)
rn=np.random.randn(1)
b=bilger_vector(rn,fuel, oxy, elements)
print(b)

[1.0000 0.1689 0.0692 0.2061 0.5558]


Now is possible to see both the coefficient matrix and result vector needed to solve the linear system Ax=b. 

In [58]:
print("A=\n",A)
print("b.transpose=\n",b)

A=
 [[1.0000 1.0000 1.0000 1.0000 1.0000 1.0000 1.0000]
 [0.0000 1.0000 0.8881 0.0000 0.5712 0.7271 0.0000]
 [0.2513 0.0000 0.1119 0.0000 0.0000 0.0000 1.0000]
 [0.7487 0.0000 0.0000 0.0000 0.4288 0.2729 0.0000]
 [0.0000 0.0000 0.0000 1.0000 0.0000 0.0000 0.0000]]
b.transpose=
 [1.0000 0.1689 0.0692 0.2061 0.5558]


## Solving the linear system

Once the system of equations is setted up it is neccesary to solve it for the species mass fractions. Unfortunately, as can be seen, this is an undetermined system (i.e. it has more unknowns thant equations) and thus, in the case it can be solved, this solution will not be unique.

In [62]:
from sympy import *